### Imports

In [1]:
# Add lemur to path
import sys
sys.path.append("../../..")
# System stuff
import pickle as pkl
import os
from IPython.display import display
# Make plotly work in notebook mode
from plotly import __version__
from plotly.offline import (download_plotlyjs,
                            init_notebook_mode,
                            plot,
                            iplot)
init_notebook_mode(connected=True)

import lemur.utils.io as lio
import lemur.eda.plotters as lpl

### View map between indexes, raw names, and regularized names

In [2]:
BASE_PATH = "../../../data"
DATASET = "bids"
dataset = lio.BIDSDataset(BASE_PATH, DATASET)
display(dataset.getSubjectMap())
display(dataset.getTaskMap())

,mine,raw
index,,
0,sub_000,NDARAA117NEJ
1,sub_001,NDARAC904DMU
2,sub_002,NDARAD481FXF
3,sub_003,NDARAJ366ZFA
4,sub_004,NDARAM277WZT
5,sub_005,NDARAM704GKZ
6,sub_006,NDARAN385MDH


,mine,raw
index,,
0,task_000,RestingState
1,task_001,SAIIT_2AFC_Block1
2,task_002,SAIIT_2AFC_Block2
3,task_003,SAIIT_2AFC_Block3
4,task_004,SurroundSupp_Block1
5,task_005,SurroundSupp_Block2
6,task_006,Video1
7,task_007,Video2
8,task_008,Video3


In [3]:
corr = lpl.CorrelationMatrixPlotter(dataset)
corr.plot(0, 0)

cov = lpl.CovarianceMatrixPlotter(dataset)
cov.plot(0, 0)
scree = lpl.ScreePlotter(dataset)
scree.plot(0, 0)

### Load data

In [ ]:

paradigms = ["RestingState",
             #"SAIIT_2AFC_Block1",
             #"SAIIT_2AFC_Block2",
             #"SAIIT_2AFC_Block3",
             #"SurroundSupp_Block1",
             #"SurroundSupp_Block2",
             "Video1",
             "Video2",
             "Video3",
             "Video4",
             "vis_learn",
             "WISC_ProcSpeed"
            ]
D = {}
used_paradigms = []
for p in paradigms:
    file_name = "%s_%s_data.pkl" % (SUBJECT, p)
    file_path = os.path.join(BASE_PATH, DATASET, SUBJECT, file_name)
    if os.path.exists(file_path):
        used_paradigms.append(p)
        with open(file_path, 'rb') as f:
            print("Loading subject %s paradigm %s" % (SUBJECT, p))
            D[p] = pkl.load(f)
            D[p] = D[p].T
    else:
        print("Subject %s did not have paradigm %s" % (SUBJECT, p))

P = {}
used_subjects = []
for s in subjects:
    file_name = "%s_%s_data.pkl" % (s, PARADIGM)
    file_path = os.path.join(BASE_PATH, DATASET, s, file_name) 
    if os.path.exists(file_path):
        with open(file_path, 'rb') as f:
            print("Loading subject %s paradigm %s" % (s, PARADIGM))
            P[s] = pkl.load(f)
            P[s] = P[s].T
            used_subjects.append(s)
    else:
        print("Subject %s did not have paradigm %s" % (s, PARADIGM))

### Set parameters

In [ ]:
p_local = {
    'paradigm': "RestingState",
    'downsample': 500,
    'disp_chans': [1, 2, 3]
}
p_global = {
    'patient_id': SUBJECT,
    'sample_freq': 500,
}

### Load lemur library

In [ ]:
from lemur.eeg_viz.spectro_spark import sparklines
for p in used_paradigms:
    p_local['paradigm'] = p
    sparklines(D, p_global, p_local)

In [ ]:
from lemur.eeg_viz.spectro_spark import signal_heatmap
for p in used_paradigms:
    p_local['paradigm'] = p
    signal_heatmap(D, p_global, p_local)
    p_local['pc_proj'] = (1, 4)
    p_local['pc_collapse'] = True
    signal_heatmap(D, p_global, p_local)
    p_local['pc_collapse'] = False
    signal_heatmap(D, p_global, p_local)

In [ ]:
from lemur.eeg_viz.spectro_spark import correlation_heatmap
from lemur.eeg_viz.spectro_spark import covariance_heatmap
for p in used_paradigms:
    p_local['paradigm'] = p
    correlation_heatmap(D, p_global, p_local)
    covariance_heatmap(D, p_global, p_local)


In [ ]:
from lemur.distance.functions import FroMetric, ErosMetric, getDistMat
from lemur.eeg_viz.spectro_spark import distance_matrix_heatmap
p_local['metric'] = FroMetric
distance_matrix_heatmap(D, p_global, p_local)

import numpy as np
def ErosMean(x):
    return ErosMetric(x, np.mean)

p_local['metric'] = ErosMean
distance_matrix_heatmap(D, p_global, p_local)

In [ ]:
from lemur.eeg_viz.spectro_spark import scree_plot
scree_plot(D, p_global, p_local)

In [ ]:
from lemur.distance.functions import FroMetric, ErosMetric, getDistMat
from lemur.eeg_viz.spectro_spark import distance_matrix_heatmap
p_local['metric'] = FroMetric
p_local['paradigm'] = used_subjects[0]
distance_matrix_heatmap(P, p_global, p_local)

import numpy as np
def ErosMean(x):
    return ErosMetric(x, np.mean)

p_local['metric'] = ErosMean
distance_matrix_heatmap(P, p_global, p_local)